In [1]:
import pandas as pd
import numpy as np
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve 
from copy import deepcopy

from scipy.stats import chi2_contingency

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.svm import SVC

from sklearn.model_selection import KFold, cross_val_score

import copy
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Setup Dataframe

In [2]:
df = pd.read_csv("Covid-19_dataset.csv",header=1)
df['is_Male'] = pd.get_dummies(df['Gender'])['MALE']#one hot encoding gender so male = #1 and female = 0
# df['is_Female'] = pd.get_dummies(df['Gender'])['FEMALE']

In [3]:
df_bloodType = pd.get_dummies(df['Blood_Group'])

# Remove unwanted features and set up One Hot Encoding

In [4]:
blacklist = ['Age',' Incubation period (days)','Blood_Group','CONTROL_blood_group','Risk_Factor','Infection_type','Gender','Place_of_isolation','Patient_In','Patient_out']
labels = ['Severeity','Outcome']
df = df.drop(blacklist,axis=1)
df

,Severeity,Outcome,Fever,Chills,Cough,Dyspnea,Anosmia_Ageusia,Loss_of_appetite,Asthenia,Headache,Muscle_ache,Cyanosis,Rhinorrhea,Sore_throat,Diarrhea,Nausea_vomiting,is_Male
0,Moderate,Recovered,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1
1,Mild,Recovered,1,0,1,0,1,1,1,1,1,0,0,1,1,1,1
2,Severe,Died,1,1,1,1,0,0,1,1,0,1,0,1,0,0,1
3,Moderate,Recovered,1,0,1,0,1,1,1,1,1,0,1,0,1,0,1
4,Moderate,Recovered,0,0,1,0,1,1,1,1,1,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,Moderate,Recovered,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1
5664,Severe,Died,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5665,Severe,Died,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1
5666,Severe,Died,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [5]:
all_columns = set(df.columns)
all_columns = all_columns - set(['Severeity','Outcome'])
all_columns = list(all_columns)
df = pd.concat([df,df_bloodType],axis=1)

# Reset cells to remove empty values

In [6]:
label_Severeity = df['Severeity'] #Severeity vs Outcome
label_Outcome = df['Outcome']
df = df.drop(['Severeity','Outcome'],axis=1)
df.dropna( inplace=True)
df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))#strings to numeric
df

,Fever,Chills,Cough,Dyspnea,Anosmia_Ageusia,Loss_of_appetite,Asthenia,Headache,Muscle_ache,Cyanosis,...,Nausea_vomiting,is_Male,A+,A-,AB+,AB-,B+,B-,O+,O-
0,1,1,1,1,1,1,1.0,1.0,1.0,0,...,0,1,1,0,0,0,0,0,0,0
1,1,0,1,0,1,1,1.0,1.0,1.0,0,...,1,1,0,0,0,0,0,0,1,0
2,1,1,1,1,0,0,1.0,1.0,0.0,1,...,0,1,1,0,0,0,0,0,0,0
3,1,0,1,0,1,1,1.0,1.0,1.0,0,...,0,1,1,0,0,0,0,0,0,0
4,0,0,1,0,1,1,1.0,1.0,1.0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,1,0,1,1,1,1,1.0,1.0,1.0,0,...,1,1,0,0,0,0,1,0,0,0
5664,1,1,1,1,1,1,1.0,1.0,1.0,1,...,1,1,0,0,0,0,1,0,0,0
5665,1,1,1,1,1,1,0.0,1.0,1.0,1,...,1,1,0,0,0,0,1,0,0,0
5666,1,1,1,1,1,1,1.0,1.0,1.0,1,...,1,1,1,0,0,0,0,0,0,0


# Chi-Square Test of independence

In [7]:
df_Chi = deepcopy(df)

In [8]:
map_mortality = {"Died":0,"Recovered":1}
label_Outcome_mapped = deepcopy(label_Outcome)
label_Outcome_mapped = label_Outcome_mapped.map(map_mortality)

In [9]:
label_Outcome.head(5)

0    Recovered
1    Recovered
2         Died
3    Recovered
4    Recovered
Name: Outcome, dtype: object

In [10]:
# label_Outcome_mapped
map_key = {1:"True",0:"False"}

for c in df_Chi.columns:
    df_Chi[c] = df_Chi[c].map(map_key)

In [11]:
df_Chi

,Fever,Chills,Cough,Dyspnea,Anosmia_Ageusia,Loss_of_appetite,Asthenia,Headache,Muscle_ache,Cyanosis,...,Nausea_vomiting,is_Male,A+,A-,AB+,AB-,B+,B-,O+,O-
0,True,True,True,True,True,True,True,True,True,False,...,False,True,True,False,False,False,False,False,False,False
1,True,False,True,False,True,True,True,True,True,False,...,True,True,False,False,False,False,False,False,True,False
2,True,True,True,True,False,False,True,True,False,True,...,False,True,True,False,False,False,False,False,False,False
3,True,False,True,False,True,True,True,True,True,False,...,False,True,True,False,False,False,False,False,False,False
4,False,False,True,False,True,True,True,True,True,False,...,False,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,True,False,True,True,True,True,True,True,True,False,...,True,True,False,False,False,False,True,False,False,False
5664,True,True,True,True,True,True,True,True,True,True,...,True,True,False,False,False,False,True,False,False,False
5665,True,True,True,True,True,True,False,True,True,True,...,True,True,False,False,False,False,True,False,False,False
5666,True,True,True,True,True,True,True,True,True,True,...,True,True,True,False,False,False,False,False,False,False


In [21]:
def rank(predictor,blood_type):

    df_temp = df[df[blood_type] == 1]
    df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
    
    

    contingency_table = pd.crosstab(df_temp['Outcome'], df_temp[predictor])

    # Apply chi-square test
    chi2, p_value, dof, expected_freq = chi2_contingency(contingency_table)
    
    
    return chi2, p_value
    
    
    





blood_types = ["A+","A-","AB+","B+","B-","O+","O-"]

dataframes = []

BloodType_df_abs = pd.DataFrame()
for blood_type in blood_types:
    print(blood_type,"-----------------------")
    list_of_AUC = []
    for feature in df.columns:
        if feature == "Blood_Group":
            continue
        
        chi2, p_value = rank(feature,blood_type)
        
        list_of_AUC.append({blood_type+" Feature":feature, blood_type+" Chi" : chi2})

        
    df_temp = pd.DataFrame(list_of_AUC)
    df_temp[blood_type+' Chi'] = abs(df_temp[blood_type+' Chi'])
    
    df_temp = df_temp.sort_values(by=[blood_type+' Chi',blood_type+' Feature'], ascending=False)
    
    df_temp = df_temp.reset_index()
    
    
    
    dataframes.append(df_temp)
    
    
    

    BloodType_df_abs[blood_type+' Feature'] = df_temp[blood_type+' Feature']
    BloodType_df_abs[blood_type+' Chi'] = df_temp[blood_type+' Chi']
    #print(df_temp)




A+ -----------------------
A- -----------------------


C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

AB+ -----------------------


C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

...

C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

O+ -----------------------


C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

O- -----------------------


C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\2228033452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [22]:
BloodType_df_abs= BloodType_df_abs[~BloodType_df_abs.isnull().any(axis=1)]
BloodType_df_abs

,A+ Feature,A+ Chi,A- Feature,A- Chi,AB+ Feature,AB+ Chi,B+ Feature,B+ Chi,B- Feature,B- Chi,O+ Feature,O+ Chi,O- Feature,O- Chi
0,Cyanosis,49.228152,Cyanosis,5.369024,Cyanosis,33.728201,Cyanosis,45.616682,Cyanosis,30.697786,Cyanosis,60.602464,Cyanosis,38.178336
1,Diarrhea,1.531517,Headache,2.595064,Rhinorrhea,0.874602,Cough,5.933919,Diarrhea,2.379438,is_Male,4.702762,Headache,1.332666
2,Asthenia,1.229929,Fever,0.571202,Asthenia,0.470804,is_Male,1.935521,Fever,0.443890,Chills,2.518146,is_Male,0.611316
3,Headache,0.449298,Cough,0.365675,Anosmia_Ageusia,0.384235,Headache,0.814115,Anosmia_Ageusia,0.398202,Anosmia_Ageusia,1.738647,Chills,0.485005
4,Chills,0.420113,Diarrhea,0.235912,is_Male,0.310056,Dyspnea,0.751459,Nausea_vomiting,0.268817,Diarrhea,1.332337,Diarrhea,0.187788
5,Sore_throat,0.381417,Nausea_vomiting,0.018300,Fever,0.136577,Chills,0.459857,Chills,0.125966,Sore_throat,0.792501,Sore_throat,0.001807
6,Muscle_ache,0.108042,is_Male,0.000000,Diarrhea,0.125076,Muscle_ache,0.436791,Sore_throat,0.070882,Nausea_vomiting,0.780150,Rhinorrhea,0.000000
7,Anosmia_Ageusia,0.053572,Sore_throat,0.000000,Cough,0.104390,Sore_throat,0.347472,Cough,0.038239,Fever,0.481429,O-,0.000000
8,is_Male,0.005867,Rhinorrhea,0.000000,Nausea_vomiting,0.100611,Anosmia_Ageusia,0.311641,Headache,0.026823,Headache,0.319507,O+,0.000000
9,Nausea_vomiting,0.001587,O-,0.000000,Dyspnea,0.040718,Fever,0.081761,is_Male,0.000705,Dyspnea,0.311957,Nausea_vomiting,0.000000


# Pearson

In [14]:
def rank(predictor,blood_type):

    df_temp = df[df[blood_type] == 1]
    df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
    
    corr_val = df_temp['Outcome'].corr(df_temp[predictor])
    
    return corr_val




blood_types = ["A+","A-","AB+","B+","B-","O+","O-"]

dataframes = []

BloodType_df_abs = pd.DataFrame()
for blood_type in blood_types:
    print(blood_type,"-----------------------")
    list_of_AUC = []
    for feature in df.columns:
        if feature == "Blood_Group":
            continue
            
        list_of_AUC.append({blood_type+" Feature":feature, blood_type+" Corr" : rank(feature,blood_type)})

        
    df_temp = pd.DataFrame(list_of_AUC)
    df_temp[blood_type+' Corr'] = abs(df_temp[blood_type+' Corr'])
    
    df_temp = df_temp.sort_values(by=[blood_type+' Corr',blood_type+' Feature'], ascending=False)
    
    df_temp = df_temp.reset_index()
    
    
    
    dataframes.append(df_temp)
    
    
    

    BloodType_df_abs[blood_type+' Feature'] = df_temp[blood_type+' Feature']
    BloodType_df_abs[blood_type+' Corr'] = df_temp[blood_type+' Corr']
    #print(df_temp)




A+ -----------------------
A- -----------------------
AB+ -----------------------
B+ -----------------------
B- -----------------------


C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\812087630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\812087630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\812087630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

O+ -----------------------
O- -----------------------


C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\812087630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\812087630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\812087630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [15]:
BloodType_df_abs= BloodType_df_abs[~BloodType_df_abs.isnull().any(axis=1)]
BloodType_df_abs

,A+ Feature,A+ Corr,A- Feature,A- Corr,AB+ Feature,AB+ Corr,B+ Feature,B+ Corr,B- Feature,B- Corr,O+ Feature,O+ Corr,O- Feature,O- Corr
0,Cyanosis,0.191124,Headache,0.394947,Cyanosis,0.280211,Cyanosis,0.175650,Cyanosis,0.447166,Cyanosis,0.212451,Cyanosis,0.860073
1,Asthenia,0.038180,Cyanosis,0.369664,Rhinorrhea,0.057374,Cough,0.067049,Diarrhea,0.131764,is_Male,0.060274,Headache,0.305936
2,Diarrhea,0.036702,Cough,0.193030,Asthenia,0.051399,is_Male,0.038172,Anosmia_Ageusia,0.115584,Chills,0.045261,is_Male,0.159239
3,Headache,0.025864,Fever,0.144479,Anosmia_Ageusia,0.049281,Dyspnea,0.031534,Fever,0.078716,Anosmia_Ageusia,0.044105,Chills,0.150481
4,Sore_throat,0.022350,Diarrhea,0.117213,Dyspnea,0.037259,Headache,0.029683,Headache,0.065705,Diarrhea,0.034714,Diarrhea,0.125264
5,Chills,0.020929,Nausea_vomiting,0.063315,is_Male,0.034234,Anosmia_Ageusia,0.022146,Nausea_vomiting,0.060365,Sore_throat,0.029760,Sore_throat,0.097534
6,Anosmia_Ageusia,0.015125,Muscle_ache,0.062151,Cough,0.030780,Muscle_ache,0.021422,Sore_throat,0.059894,Dyspnea,0.027354,Muscle_ache,0.087781
7,Muscle_ache,0.013801,Rhinorrhea,0.062151,Fever,0.028949,Chills,0.020750,Cough,0.056953,Nausea_vomiting,0.026825,Rhinorrhea,0.078909
8,Dyspnea,0.010632,Asthenia,0.056960,Diarrhea,0.026463,Sore_throat,0.020600,Chills,0.052152,Fever,0.023686,Anosmia_Ageusia,0.059351
9,Rhinorrhea,0.006928,Sore_throat,0.046204,Nausea_vomiting,0.023284,Fever,0.012453,Asthenia,0.043965,Headache,0.023469,Cough,0.053797


# Spearman

In [16]:
def rank(predictor,blood_type):

    df_temp = df[df[blood_type] == 1]
    df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
    
    corr_val = df_temp['Outcome'].corr(df_temp[predictor],method='spearman')
    
    return corr_val




blood_types = ["A+","A-","AB+","B+","B-","O+","O-"]

dataframes = []

BloodType_df_abs = pd.DataFrame()
for blood_type in blood_types:
    print(blood_type,"-----------------------")
    list_of_AUC = []
    for feature in df.columns:
        if feature == "Blood_Group":
            continue
            
        list_of_AUC.append({blood_type+" Feature":feature, blood_type+" Corr" : rank(feature,blood_type)})

        
    df_temp = pd.DataFrame(list_of_AUC)
    df_temp[blood_type+' Corr'] = abs(df_temp[blood_type+' Corr'])
    
    df_temp = df_temp.sort_values(by=[blood_type+' Corr',blood_type+' Feature'], ascending=False)
    
    df_temp = df_temp.reset_index()
    
    
    
    dataframes.append(df_temp)
    
    
    

    BloodType_df_abs[blood_type+' Feature'] = df_temp[blood_type+' Feature']
    BloodType_df_abs[blood_type+' Corr'] = df_temp[blood_type+' Corr']
    #print(df_temp)




C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

A+ -----------------------


C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

A- -----------------------
AB+ -----------------------
B+ -----------------------
B- -----------------------


C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is 

O+ -----------------------
O- -----------------------


C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Outcome'] = label_Outcome_mapped[df[blood_type] == 1]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
C:\Users\Rohan\AppData\Local\Temp\ipykernel_26772\3637240226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d

In [17]:
BloodType_df_abs= BloodType_df_abs[~BloodType_df_abs.isnull().any(axis=1)]
BloodType_df_abs

,A+ Feature,A+ Corr,A- Feature,A- Corr,AB+ Feature,AB+ Corr,B+ Feature,B+ Corr,B- Feature,B- Corr,O+ Feature,O+ Corr,O- Feature,O- Corr
0,Cyanosis,0.191124,Headache,0.394947,Cyanosis,0.280211,Cyanosis,0.175650,Cyanosis,0.447166,Cyanosis,0.212451,Cyanosis,0.860073
1,Asthenia,0.038180,Cyanosis,0.369664,Rhinorrhea,0.057374,Cough,0.067049,Diarrhea,0.131764,is_Male,0.060274,Headache,0.305936
2,Diarrhea,0.036702,Cough,0.193030,Asthenia,0.051399,is_Male,0.038172,Anosmia_Ageusia,0.115584,Chills,0.045261,is_Male,0.159239
3,Headache,0.025864,Fever,0.144479,Anosmia_Ageusia,0.049281,Dyspnea,0.031534,Fever,0.078716,Anosmia_Ageusia,0.044105,Chills,0.150481
4,Sore_throat,0.022350,Diarrhea,0.117213,Dyspnea,0.037259,Headache,0.029683,Headache,0.065705,Diarrhea,0.034714,Diarrhea,0.125264
5,Chills,0.020929,Nausea_vomiting,0.063315,is_Male,0.034234,Anosmia_Ageusia,0.022146,Nausea_vomiting,0.060365,Sore_throat,0.029760,Sore_throat,0.097534
6,Anosmia_Ageusia,0.015125,Rhinorrhea,0.062151,Cough,0.030780,Muscle_ache,0.021422,Sore_throat,0.059894,Dyspnea,0.027354,Muscle_ache,0.087781
7,Muscle_ache,0.013801,Muscle_ache,0.062151,Fever,0.028949,Chills,0.020750,Cough,0.056953,Nausea_vomiting,0.026825,Rhinorrhea,0.078909
8,Dyspnea,0.010632,Asthenia,0.056960,Diarrhea,0.026463,Sore_throat,0.020600,Chills,0.052152,Fever,0.023686,Anosmia_Ageusia,0.059351
9,Rhinorrhea,0.006928,Sore_throat,0.046204,Nausea_vomiting,0.023284,Fever,0.012453,Asthenia,0.043965,Headache,0.023469,Fever,0.053797


# Spearman